# 音MAD作者支援用簡易ノートブック

音MAD製作において便利な機能をつめこんだノートブックです。 

Google Drive上のsiendirディレクトリにダウンロードした動画や音声などを格納します。 

多少設計にグダグダなところがありますが、動けばok！！の精神で見逃してください…

## Google Driveマウント
Google Driveをマウントします。 

必ず最初に実行してください。

In [ ]:
#@title Google Driveをマウント
from google.colab import drive
import os
drive.mount('/content/drive')

if not os.path.exists("/content/drive/MyDrive/siendir"):
  !mkdir /content/drive/MyDrive/siendir


Mounted at /content/drive


## Youtube動画ダウンロード
Youtubeの動画をダウンロードし、Google Driveに保存します

デフォルトで最高画質&フレームレートでダウンロードする仕様です。 

サイトが対応しているようであれば他サイトもダウンロード可能です。

In [ ]:
#@title 初期化処理(初めに実行してください)
!apt install ffmpeg
!pip install yt-dlp

In [ ]:
#@title 動画をダウンロード(Youtube等)

movie_url = "https://www.youtube.com/watch?v=KyjhLRymzsk"#@param {type:"string"}

!yt-dlp -f "bestvideo[ext=mp4]+bestaudio[ext=m4a]" --merge-output-format mp4 -o "/content/drive/MyDrive/siendir/%(title)s-%(height)s.%(ext)s" --no-mtime "$movie_url"

In [ ]:
#@title 動画をダウンロード(ニコニコ)

movie_url = "https://www.nicovideo.jp/watch/sm39300654"#@param {type:"string"}

!yt-dlp -f "b" --merge-output-format mp4 -o "/content/drive/MyDrive/siendir/%(title)s-%(height)s.%(ext)s" --no-mtime "$movie_url"

## 動画形式変換
動画のファイル形式を変換したり、音声にしたりできるものです

Twitterにアップロードなどする際にコーデックの問題でアップロードできなかったり、簡単に音声ファイルに変換できたりするとうれしいと思い作成

In [ ]:
#@title 初期化処理(初めに実行してください)
!apt install ffmpeg

In [ ]:
#@title 互換性のある動画形式(H264+AAC)に変換

import glob

files = glob.glob('/content/drive/My Drive/siendir/*.mp4', recursive=True)

for index, tmpname in enumerate(files):
  print("{}. {}".format(index, tmpname))

num = int(input("変換したい動画を選択してください: "))
filename = files[num]
filename_after = "{}_convert.mp4".format(files[num].split(".")[0])


!ffmpeg -i "$filename" -vcodec h264_nvenc  -acodec aac "$filename_after"


In [ ]:
#@title mp3形式に変換
import glob

files = glob.glob('/content/drive/My Drive/siendir/*.mp4', recursive=True) + glob.glob('/content/drive/My Drive/siendir/*.wav', recursive=True)

for index, tmpname in enumerate(files):
  print("{}. {}".format(index, tmpname))

num = int(input("変換したい動画を選択してください: "))
filename = files[num]
filename_after = "{}_convert.mp3".format(files[num].split(".")[0])


!ffmpeg -i "$filename" -acodec libmp3lame "$filename_after"

In [ ]:
#@title wav形式に変換
import glob

files = glob.glob('/content/drive/My Drive/siendir/*.mp4', recursive=True) + glob.glob('/content/drive/My Drive/siendir/*.mp3', recursive=True)

for index, tmpname in enumerate(files):
  print("{}. {}".format(index, tmpname))

num = int(input("変換したい動画を選択してください: "))
filename = files[num]
filename_after = "{}_convert.wav".format(files[num].split(".")[0])


!ffmpeg -i "$filename" -acodec pcm_s24le "$filename_after"

## 音源分離(Demucs v3)
音源分離をし、局から歌声を抽出したりBGMを除去したりするものです  

こちらはDemucs v3を用いるため、かなりの精度が期待できます  

4音に分割する仕様です  

分離したい音源は事前に上で音声ファイルに変換しておいてください  

In [ ]:
#@title 初期化処理(初めに実行してください)
#@markdown また、これを実行したあとは「RESTART RUNTIME」ボタンを押す必要があるっぽいです
!apt install ffmpeg
!pip install demucs

In [ ]:
#@title 音源分離

import glob
import os

files = glob.glob('/content/drive/My Drive/siendir/*.wav', recursive=True) + glob.glob('/content/drive/My Drive/siendir/*.mp3', recursive=True)

for index, tmpname in enumerate(files):
  print("{}. {}".format(index, tmpname))

num = int(input("音源分離したい音声を選択してください: "))
filename = files[num]
dirname_after = "{}".format(files[num].split(".")[0])

if not os.path.exists(dirname_after):
  !mkdir "$dirname_after"

!demucs -o "$dirname_after" -n "mdx_extra" "$filename"

## 音源分離(spleeter)
音源分離をし、曲から歌声を抽出したりBGMを除去したりするものです 

こちらはspleeterを用いるため、少し精度は劣ります  

5音に分割する仕様です

分離したい音源は事前に上で音声ファイルに変換しておいてください


In [ ]:
#@title 初期化処理(初めに実行してください)
#@markdown また、これを実行したあとは「RESTART RUNTIME」ボタンを押す必要があるっぽいです
!pip install spleeter-gpu

In [ ]:
#@title 音源分離

import glob
import os

files = glob.glob('/content/drive/My Drive/siendir/*.wav', recursive=True) + glob.glob('/content/drive/My Drive/siendir/*.mp3', recursive=True)

for index, tmpname in enumerate(files):
  print("{}. {}".format(index, tmpname))

num = int(input("音源分離したい音声を選択してください: "))
filename = files[num]
dirname_after = "{}".format(files[num].split(".")[0])

if not os.path.exists(dirname_after):
  !mkdir "$dirname_after"

!spleeter separate -o "$dirname_after" -p spleeter:5stems -i "$filename"

## 動画高画質化  

動画を高画質にするものです  

デフォルトでは二倍の解像度になります  

Real-ESRGANという技術を用いています  


In [ ]:
#@title 初期化処理(初めに実行してください)
!apt install ffmpeg
!git clone https://github.com/xinntao/Real-ESRGAN
%cd Real-ESRGAN

!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

%cd ../

In [ ]:
#@title 高画質素材を作成

import glob
import os

files = glob.glob('/content/drive/My Drive/siendir/*.mp4', recursive=True)

for index, tmpname in enumerate(files):
  print("{}. {}".format(index, tmpname))

num = int(input("変換したい動画を選択してください: "))
filename = files[num]

dirname_before_tmp = "{}_beforeesr".format(files[num].split(".")[0])
audioname_before_tmp = "{}_beforeesr/audio.wav".format(files[num].split(".")[0])
imgname_before_tmp = "{}_beforeesr/%04d.png".format(files[num].split(".")[0])
dirname_after_tmp = "{}_afteresr".format(files[num].split(".")[0])
imgname_after_tmp = "{}_afteresr/%04d_out.png".format(files[num].split(".")[0])

filename_after = "{}_esr.mp4".format(files[num].split(".")[0])

if not os.path.exists(dirname_before_tmp):
  !mkdir "$dirname_before_tmp"

!ffmpeg -i "$filename" "$audioname_before_tmp"
!ffmpeg -i "$filename" -vcodec png "$imgname_before_tmp"

%cd Real-ESRGAN
!python ./inference_realesrgan.py -n  RealESRGAN_x4plus -i "$dirname_before_tmp" -o "$dirname_after_tmp" --outscale 2
%cd ../

!ffmpeg -i "$imgname_after_tmp" -i "$audioname_before_tmp" "$filename_after"


## 透過素材作成

背景が透過された素材を作成するものです 

anime-segmentationという技術を用いています  

また最終的に作成する映像はUtVideo RGBA形式で出力されます 

インストールされていればAviutlでも読み込めるようになっているはずです 

対応していない場合はGoogle Driveのディレクトリから連番pngをダウンロードすれば大丈夫だと思います


In [ ]:
#@title 初期化処理(初めに実行してください)
!apt install ffmpeg
!git clone https://github.com/SkyTNT/anime-segmentation
!pip install -r anime-segmentation/requirements.txt
!wget -P anime-segmentation https://huggingface.co/skytnt/anime-seg/resolve/main/isnetis.ckpt

In [ ]:
#@title 透過素材を作成

import glob
import os

files = glob.glob('/content/drive/My Drive/siendir/*.mp4', recursive=True)

for index, tmpname in enumerate(files):
  print("{}. {}".format(index, tmpname))

num = int(input("変換したい動画を選択してください: "))
filename = files[num]

dirname_before_tmp = "{}_beforeseg".format(files[num].split(".")[0])
audioname_before_tmp = "{}_beforeseg/audio.wav".format(files[num].split(".")[0])
imgname_before_tmp = "{}_beforeseg/%04d.png".format(files[num].split(".")[0])
dirname_after_tmp = "{}_afterseg".format(files[num].split(".")[0])
imgname_after_tmp = "{}_afterseg/%06d.png".format(files[num].split(".")[0])

filename_after = "{}_seg.avi".format(files[num].split(".")[0])

if not os.path.exists(dirname_before_tmp):
  !mkdir "$dirname_before_tmp"

!ffmpeg -i "$filename" "$audioname_before_tmp"
!ffmpeg -i "$filename" -vcodec png "$imgname_before_tmp"

!python ./anime-segmentation/inference.py --net isnet_is --ckpt ./anime-segmentation/isnetis.ckpt --data "$dirname_before_tmp" --out "$dirname_after_tmp" --img-size 1024 --only-matted

!ffmpeg -i "$imgname_after_tmp" -i "$audioname_before_tmp" -vcodec utvideo "$filename_after"
